TASK1

In [56]:
import random
import pandas as pd


attendance_raw = []

cohort = ['alpha','beta','gamma']

for i in range (1,25):
    attendance_raw.append({
        'student_id': f"S{i:03}",
        'cohort': random.choice(cohort),
        'attended_sessions': random.randint(0,6),
        'expected_sessions': 6
    })

attendance = pd.DataFrame(attendance_raw)

print(attendance.head(5))

attendance.info()

  student_id cohort  attended_sessions  expected_sessions
0       S001  alpha                  0                  6
1       S002   beta                  0                  6
2       S003  gamma                  2                  6
3       S004  gamma                  4                  6
4       S005   beta                  5                  6
<class 'pandas.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   student_id         24 non-null     str  
 1   cohort             24 non-null     str  
 2   attended_sessions  24 non-null     int64
 3   expected_sessions  24 non-null     int64
dtypes: int64(2), str(2)
memory usage: 900.0 bytes


TASK2

In [57]:
attendance_indexed = attendance.set_index('student_id')

excused_absences = pd.Series([1,2,1,2,1,0,0,2,1,0],
                             index = ['S001',
                                      'S002',
                                      'S010',
                                      'S015',
                                      'S030',
                                      'S005',
                                      'S008',
                                      'S020',
                                      'S025',
                                      'S003'])




In [58]:
print(attendance_indexed.head())


           cohort  attended_sessions  expected_sessions
student_id                                             
S001        alpha                  0                  6
S002         beta                  0                  6
S003        gamma                  2                  6
S004        gamma                  4                  6
S005         beta                  5                  6


In [59]:
attendance_indexed['adjusted_attendance'] = attendance_indexed['attended_sessions'] + excused_absences


In [60]:
print(attendance_indexed[['attended_sessions','adjusted_attendance']])


            attended_sessions  adjusted_attendance
student_id                                        
S001                        0                  1.0
S002                        0                  2.0
S003                        2                  2.0
S004                        4                  NaN
S005                        5                  5.0
S006                        1                  NaN
S007                        1                  NaN
S008                        2                  2.0
S009                        1                  NaN
S010                        4                  5.0
S011                        1                  NaN
S012                        2                  NaN
S013                        1                  NaN
S014                        3                  NaN
S015                        0                  2.0
S016                        5                  NaN
S017                        3                  NaN
S018                        6  

In [61]:
attendance_indexed['adjusted_attendance'] = attendance_indexed['adjusted_attendance'].fillna(
    attendance_indexed['attended_sessions']
)

In [62]:
print(attendance_indexed[['attended_sessions', 'adjusted_attendance']])

            attended_sessions  adjusted_attendance
student_id                                        
S001                        0                  1.0
S002                        0                  2.0
S003                        2                  2.0
S004                        4                  4.0
S005                        5                  5.0
S006                        1                  1.0
S007                        1                  1.0
S008                        2                  2.0
S009                        1                  1.0
S010                        4                  5.0
S011                        1                  1.0
S012                        2                  2.0
S013                        1                  1.0
S014                        3                  3.0
S015                        0                  2.0
S016                        5                  5.0
S017                        3                  3.0
S018                        6  

TASK3

In [63]:
for i in range(random.randint(1, 5)):
    row_index = random.choice(range(1,len(attendance)))
    current = attendance.loc[row_index, 'cohort']
    messed_value = current.upper() if random.random() < 0.75 else f"    {current}  "
    attendance.loc[row_index, 'cohort'] = messed_value


In [64]:
attendance

,student_id,cohort,attended_sessions,expected_sessions
0,S001,alpha,0,6
1,S002,beta,0,6
2,S003,gamma,2,6
3,S004,gamma,4,6
4,S005,beta,5,6
5,S006,alpha,1,6
6,S007,beta,1,6
7,S008,alpha,2,6
8,S009,alpha,1,6
9,S010,beta,4,6


In [65]:
attendance['cohort'] = attendance['cohort'].str.strip().str.lower()

attendance['cohort'].unique()

<StringArray>
['alpha', 'beta', 'gamma']
Length: 3, dtype: str

TASK4

In [66]:
low_attendance = attendance[attendance['attended_sessions'] < attendance['expected_sessions']]

average_by_cohort = attendance.groupby('cohort')['attended_sessions'].mean()


In [67]:
print(f"""Summary: 
Low attendances: {low_attendance}
Averages: {average_by_cohort}
Verifying that cohorts in the summary match the cleaned ones: \n{average_by_cohort.index.tolist()}""")

Summary: 
Low attendances:    student_id cohort  attended_sessions  expected_sessions
0        S001  alpha                  0                  6
1        S002   beta                  0                  6
2        S003  gamma                  2                  6
3        S004  gamma                  4                  6
4        S005   beta                  5                  6
5        S006  alpha                  1                  6
6        S007   beta                  1                  6
7        S008  alpha                  2                  6
8        S009  alpha                  1                  6
9        S010   beta                  4                  6
10       S011  gamma                  1                  6
11       S012  gamma                  2                  6
12       S013  gamma                  1                  6
13       S014  alpha                  3                  6
14       S015  alpha                  0                  6
15       S016   beta         

TASK5

In [68]:
attendance['attendance_ok'] = attendance['attended_sessions'] >= attendance['expected_sessions']

low_attendance = attendance[attendance['attended_sessions'] < attendance['expected_sessions']]


In [69]:
validation = (low_attendance['attendance_ok'] == False).all()

validation

np.True_